# 🔬 Advanced Breast Cancer Classification - BUSI Dataset
## DenseNet121 Optimization (Target: >96% Macro-F1)

**Optimizations:**
- Progressive Fine-Tuning (freeze 80% → unfreeze top 20%)
- Focal Loss (γ=2) for class imbalance
- CLAHE + Mixup/CutMix augmentation
- CBAM attention mechanism
- Cosine Annealing LR

**GPU Required:** T4 or better (Runtime → Change runtime type → GPU)

In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

## 📦 Setup & Installation

In [ ]:
# Install dependencies
!pip install -q umap-learn shap tabulate opencv-python

In [ ]:
# Clone repository (or upload files)
# !git clone <your_repo_url>
# %cd moussokene_master_search

# Or upload scripts manually
from google.colab import files
import os

# Create directories
!mkdir -p scripts models results logs datasets/train datasets/test

## 📊 Download BUSI Dataset

In [ ]:
# Option 1: Download from Kaggle
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# Upload your kaggle.json here
# !kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset
# !unzip -q breast-ultrasound-images-dataset.zip -d datasets/

# Option 2: Upload manually
# files.upload()  # Upload your dataset zip

# Verify dataset structure
!ls -R datasets/

## 🔧 Upload Script Files

Upload these files to `scripts/` directory:
- `focal_loss.py`
- `augmentation.py`
- `cbam.py`
- `train_advanced.py`
- `compare_models.py`
- `ablation_study.py`
- `visualize_gradcam.py`
- `visualize_all.py`

In [ ]:
# Upload scripts
%cd scripts
uploaded = files.upload()
%cd ..

## 🚀 Training: DenseNet121 Advanced

In [ ]:
%cd scripts

# Import and run training
from train_advanced import train_model, CONFIG

# Adjust config for Colab (optional)
CONFIG['batch_size'] = 16
CONFIG['initial_epochs'] = 15
CONFIG['fine_tune_epochs'] = 25

# Train model
model, results, history = train_model(
    model_name='densenet121',
    train_dir='../datasets/train/',
    val_dir='../datasets/test/',
    config=CONFIG
)

print(f"\n{'='*80}")
print(f"TRAINING COMPLETE!")
print(f"Final Accuracy: {results['accuracy']:.4f}")
print(f"Final Macro-F1: {results['macro_f1']:.4f}")
print(f"{'='*80}\n")

## 📊 View Training Results

In [ ]:
# Display training history plot
from IPython.display import Image, display
import json

# Show training curves
display(Image('../results/densenet121_training_history.png'))

# Show confusion matrix
display(Image('../results/densenet121_confusion_matrix.png'))

# Print detailed results
with open('../results/densenet121_results.json', 'r') as f:
    results = json.load(f)
    print(json.dumps(results['classification_report'], indent=2))

## 🔄 Model Comparison (DenseNet vs ResNet vs EfficientNet)

In [ ]:
from compare_models import compare_models

# Train and compare all models
all_results = compare_models(
    train_dir='../datasets/train/',
    val_dir='../datasets/test/'
)

# Display comparison table
import pandas as pd
df = pd.read_csv('../results/model_comparison.csv')
print(df.to_markdown(index=False))

# Show comparison charts
display(Image('../results/models_comparison_charts.png'))

## 🧪 Ablation Study

In [ ]:
from ablation_study import run_ablation_study

# Run ablation experiments
ablation_results = run_ablation_study(
    train_dir='../datasets/train/',
    val_dir='../datasets/test/'
)

# Display results
df_ablation = pd.read_csv('../results/ablation_study.csv')
print(df_ablation.to_markdown(index=False))

# Show ablation plot
display(Image('../results/ablation_study_plot.png'))

## 🎨 Grad-CAM Visualizations

In [ ]:
from visualize_gradcam import generate_gradcam_report

class_names = ['benign', 'malignant', 'normal']

# Generate Grad-CAM for DenseNet121
generate_gradcam_report(
    model_path='../models/densenet121_final.keras',
    data_dir='../datasets/test/',
    class_names=class_names,
    n_samples=12
)

# Display Grad-CAM results
display(Image('../results/densenet121_final_gradcam.png'))
display(Image('../results/densenet121_final_feature_maps.png'))

## 📈 Advanced Visualizations (t-SNE, UMAP, SHAP)

In [ ]:
from visualize_all import generate_all_visualizations

# Generate all visualizations
generate_all_visualizations(
    model_path='../models/densenet121_final.keras',
    data_dir='../datasets/test/',
    class_names=class_names
)

# Display results
print("\n=== t-SNE/UMAP Embeddings ===")
display(Image('../results/densenet121_final_embeddings.png'))

print("\n=== ROC Curves ===")
display(Image('../results/densenet121_final_roc_curves.png'))

print("\n=== Detailed Confusion Matrix ===")
display(Image('../results/densenet121_final_confusion_detailed.png'))

## 🎯 Ensemble Prediction

In [ ]:
from ablation_study import test_ensemble

# Test ensemble of all models
model_paths = [
    '../models/densenet121_final.keras',
    '../models/resnet50_final.keras',
    '../models/efficientnetb0_final.keras'
]

ensemble_results = test_ensemble(
    model_paths=model_paths,
    val_dir='../datasets/test/'
)

print(f"\nEnsemble Accuracy: {ensemble_results['accuracy']:.4f}")
print(f"Ensemble Macro-F1: {ensemble_results['macro_f1']:.4f}")

## 💾 Download Results

In [ ]:
# Zip all results
!zip -r results.zip ../results/ ../models/

# Download
files.download('results.zip')

## 🔮 Predict on New Image

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load model
model = tf.keras.models.load_model('../models/densenet121_final.keras', compile=False)

# Upload test image
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# Preprocess
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
class_names = ['benign', 'malignant', 'normal']
predicted_class = class_names[np.argmax(predictions)]
confidence = np.max(predictions) * 100

# Display
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title(f'Prediction: {predicted_class.upper()} ({confidence:.2f}%)', fontsize=14)
plt.axis('off')
plt.show()

print(f"\nPrediction Probabilities:")
for i, class_name in enumerate(class_names):
    print(f"  {class_name.capitalize()}: {predictions[0][i]*100:.2f}%")

## 📊 TensorBoard (Optional)

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Launch TensorBoard
%tensorboard --logdir ../logs/

---

## 📚 References

1. **DenseNet**: Huang et al., CVPR 2017 - [arXiv:1608.06993](https://arxiv.org/abs/1608.06993)
2. **Focal Loss**: Lin et al., ICCV 2017 - [arXiv:1708.02002](https://arxiv.org/abs/1708.02002)
3. **Mixup**: Zhang et al., ICLR 2018 - [arXiv:1710.09412](https://arxiv.org/abs/1710.09412)
4. **CBAM**: Woo et al., ECCV 2018 - [arXiv:1807.06521](https://arxiv.org/abs/1807.06521)
5. **Grad-CAM**: Selvaraju et al., ICCV 2017 - [arXiv:1610.02391](https://arxiv.org/abs/1610.02391)

**Dataset**: BUSI - Al-Dhabyani et al., 2020

---

**⚠️ Disclaimer**: This system is for research purposes only. Not for clinical diagnosis.